# Synonym expansion playground

In [1]:
from cpr_sdk.search_adaptors import VespaSearchAdapter
from cpr_sdk.utils import dig

/Users/kalyan/Library/Caches/pypoetry/virtualenvs/cpr-sdk-MbEuBZqr-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Setting up and running our current YQL query

In [2]:
VESPA_URL = "http://localhost:8080"
search_adapter = VespaSearchAdapter(VESPA_URL)

In [90]:
# this doesn't work !!


def do_synonym_search_query(query_string: str):
    if "ndc" in query_string:
        query_string_modified = query_string.replace(
            "ndc", "nationally determined contribution"
        )
        family_yql = f"""select * from sources family_document, document_passage where weakAnd(userInput(\"{query_string}\"), userInput(\"{query_string_modified}\")) """
    else:
        family_yql = f"""select * from sources family_document, document_passage where (userInput(@query_string)) """

    print(family_yql)

    return search_adapter.client.query(
        {
            "yql": family_yql,
            "query_string": query_string,
            "rules.off": False,
            "rules.rulebase": "synonyms_acronyms",
        }
    ).json


do_synonym_search_query("ndc")

select * from sources family_document, document_passage where weakAnd(userInput("ndc"), userInput("nationally determined contribution")) 


{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 177},
  'coverage': {'coverage': 100,
   'documents': 1968,
   'full': True,
   'nodes': 2,
   'results': 2,
   'resultsFull': 2},
  'children': [{'id': 'id:doc_search:document_passage::CCLW.executive.4934.1571.15',
    'relevance': 0.08982502681077315,
    'source': 'family-document-passage',
    'fields': {'sddocname': 'document_passage',
     'documentid': 'id:doc_search:document_passage::CCLW.executive.4934.1571.15',
     'text_block_not_stemmed': 'The contributions from development partners and non-government organizations and the private\nsector are considered vital to meeting our national goals, and they have done so much over the past\nyears. Our national visions and goals cannot be met without the commitment, the partnerships and\nthe willingness of all the Marshallese people.',
     'search_weights_ref': 'id:doc_search:search_weights::default_weights',
     'family_document_ref': 'id:doc_search:family_

In [46]:
all_family_yql = """select * from sources family_document where true"""
search_adapter.client.query(
    {
        "yql": all_family_yql,
        "query_string": "*",
    }
).json

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 3},
  'coverage': {'coverage': 100,
   'documents': 3,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'id:doc_search:family_document::CCLW.executive.10014.4470',
    'relevance': 0.0,
    'source': 'family-document-passage',
    'fields': {'sddocname': 'family_document',
     'documentid': 'id:doc_search:family_document::CCLW.executive.10014.4470',
     'search_weights_ref': 'id:doc_search:search_weights::default_weights',
     'family_name': 'Environmental Strategy for 2014-2023',
     'family_description': 'The Environment Strategy for Moldova for 2014-2023 was adopted by the Moldovan Government via decision No. 301 of 24.04.2014. The second of the strategy\'s key objectives includes the integration of "environmental protection, sustainable development, and green economy principles, of climate change adaptation principles into all sectors of the national economy". 